In [3]:
import numpy as np
import pandas as pd
import librosa
import os
import glob
import re
import seaborn as sn
import matplotlib.pyplot as plt
import scipy
import time
import collections
import keras
import random
from keras.utils import np_utils
from keras.layers import MaxPooling1D, Conv1D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, LSTM, ELU, Bidirectional, Attention
from keras.layers import Dense, Dropout, Activation, Flatten, CuDNNLSTM
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import tensorflow as tf
import pickle
from keras.utils.vis_utils import plot_model
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
import keras_tuner
from keras import backend as K
import shutil
import os
import tensorflow_addons as tfa
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from sklearn import metrics

2023-05-03 14:00:28.120270: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 14:00:35.174190: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib:/home/stenmarken/miniconda3/envs/tensorflow/lib/
2023-05-03 14:00:35.174291: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib:/home/stenmarken/miniconda3/envs/tensorfl

### Global variables

In [4]:
global_model_type = 'tune_cnn'
global_folder_name = 'weyo4'
file_suffix = 'weyo1'
global_batch_size = 32
global_epochs = 50
global_learning_rate = 0.0001
own_file_path = os.getcwd() 
global_hyperparameter_folder_name = 'real_cnn_filter_size_dropout1' #lstm_filter_size_dropout1' #'cnn_filter_size_dropout'

In [5]:
def instrument_code(filename):
    """
    Function that takes in a filename and returns instrument based on naming convention
    """
    # Synth lead borttagen. id = 9
    class_names=['bass', 'brass', 'flute', 'guitar', 
             'keyboard', 'mallet', 'organ', 'reed', 
             'string', 'vocal']
    
    for name in class_names:
        if name in filename:
            return class_names.index(name)
    else:
        return None

## Get data

In [6]:
def get_data_from_file(file):
    with open(file, "rb") as f:
        X_train_full  = pickle.load(f)

    X_train = []
    y_train = []

    for (key, value) in X_train_full:
        X_train.append(value)
        y_train.append(instrument_code(key))

    X_train_numpy = np.array(X_train)
    y_train_numpy = np.array(y_train)
    return (X_train_numpy, y_train_numpy)

In [7]:
def get_old_test_data(file):
    with open(file, 'rb') as f:
        old_X_test_full = pickle.load(f)

    old_X_test = []
    old_y_test = []

    for(key, value) in old_X_test_full.items():
        #temporal_value = np.mean(value, axis = 1)
        old_X_test.append(value)
        old_y_test.append(instrument_code(key))
        
    old_y_test_numpy = np.asarray(old_y_test)
    old_X_test_numpy = np.asarray(old_X_test)
    return (old_X_test_numpy, old_y_test_numpy)

### Calculate metrics

In [8]:
def calc_metrics(y_true, y_pred):
    print(metrics.classification_report(y_true, y_pred, digits=3))

In [9]:
def hyperparameter_cnn_model(hp):
    """
    Function that builds a CNN model with optimized hyperparameters including filter size and dropout rate
    """

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), 
                    activation='relu', padding='same', input_shape=(126, 13, 1)))
    model.add(Conv2D(32, kernel_size=(3, 3), 
                    activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Conv2D(filters = hp.Choice('conv_2_filters_1', values = [32, 64, 128]), kernel_size=(3, 3),
                    activation='relu', padding='same'))
    model.add(Conv2D(filters = hp.Choice('conv_2_filters_1', values = [32, 64, 128]), kernel_size=(3, 3),
                    activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Conv2D(filters = hp.Choice('conv_2_filters_2', values = [32, 64, 128, 256]), kernel_size=(3, 3),
                    activation='relu', padding='same'))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=global_learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [10]:
def hyperparameter_lstm_model(hp):
    """
    Function that builds a LSTM model with optimized hyperparameters including filter size and dropout rate
    """
    
    model = Sequential()
    #model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True), input_shape=(126, 128)))
    #model.add(CuDNNLSTM(4, input_shape=(126, 128), return_sequences=True))
    #model.add(Dropout(0.5))
    model.add(LSTM(32, input_shape=(126, 13), return_sequences=True))
    model.add(Dropout(hp.Float('dropout_lstm', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(units = hp.Choice('lstm_layer_units_1', values = [32, 64, 128]), return_sequences=True))
    model.add(Dropout(hp.Float('dropout_lstm', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(units = hp.Choice('lstm_layer_units_1', values = [32, 64, 128]), return_sequences=True))
    model.add(Dropout(hp.Float('dropout_lstm', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(units = hp.Choice('lstm_layer_units_2', values = [32, 64, 128]), return_sequences=True))
    model.add(Dropout(hp.Float('dropout_lstm', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(units = hp.Choice('lstm_layer_units_2', values = [32, 64, 128]), return_sequences=True))
    model.add(Dropout(hp.Float('dropout_lstm', min_value=0.1, max_value=0.5, step=0.1)))
    #model.add(LSTM(16, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=global_learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
def build_cnn_model():    
    input_shape = (126, 13, 1)
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128,(3, 3), padding='same', activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model

In [12]:
def build_lstm_model():
    model = Sequential()
    #model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True), input_shape=(126, 128)))
    #model.add(CuDNNLSTM(4, input_shape=(126, 128), return_sequences=True))
    #model.add(Dropout(0.5))
    model.add(LSTM(32, input_shape=(126, 13), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32, return_sequences=True))
    #model.add(LSTM(16, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model

In [13]:
def plot_acc_loss(history, acc_file_path, loss_file_path):
    # Taget från https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    # summarize history for accuracy
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(acc_file_path)
    plt.show()

    # summarize history for loss
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(loss_file_path)
    plt.show()

In [14]:
    
def plt_confusion_matrix(model, X_test, y_test, confusion_matrix_file_path):    
    y_prediction = model.predict(X_test)
    y_prediction = np.argmax(y_prediction, axis = 1)
    result = confusion_matrix(y_test, y_prediction , normalize='pred')

    df_cm = pd.DataFrame(result, index = [i for i in ["bass", "brass", "flute", "guitar", "keyboard", "mallet", "organ", "reed", "string", "vocal"]],
                    columns = [i for i in ["bass", "brass", "flute", "guitar", "keyboard", "mallet", "organ", "reed", "string", "vocal"]])

    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig(confusion_matrix_file_path)
    plt.show()

In [15]:
def run_cnn(fold_index):

    old_test_file = 'CustomDataFull/testdata2000.pkl'

    train_file = 'folds/' + str(fold_index) + 'train_' + file_suffix
    test_file = 'folds/' + str(fold_index) + 'test_' + file_suffix
    valid_file = 'folds/' + str(fold_index) + 'valid_' + file_suffix

    X_train, y_train = get_data_from_file(train_file)
    X_test, y_test = get_data_from_file(test_file)
    X_valid, y_valid = get_data_from_file(valid_file)
    old_X_test, old_y_test = get_old_test_data(old_test_file)

    fold_path = 'results/cnn_results/' + global_folder_name + '/' + str(fold_index) + file_suffix + '/'
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)

    model = build_cnn_model()
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=global_learning_rate), metrics=['accuracy'])
    model.summary()
    history = model.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid), batch_size=global_batch_size, epochs=global_epochs, shuffle=True, verbose=2)
    model.save(fold_path + str(fold_index) + 'model_' + file_suffix + '.h5')

    history_name = fold_path + str(fold_index) + 'history_' + file_suffix + '.history'

    with open(history_name, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

    scores = model.evaluate(X_test, y_test, verbose=1)
    y_prediction = model.predict(X_test)
    y_prediction = np.argmax(y_prediction, axis = 1)

    print("\n\ny_test : ", y_test)
    print("\n\ny_prediction : ", y_prediction)

    f1_scores = metrics.classification_report(y_test, y_prediction, digits=3)

    original_scores = model.evaluate(old_X_test, old_y_test, verbose=1)
    original_y_prediction = model.predict(old_X_test)
    original_y_prediction = np.argmax(original_y_prediction, axis = 1)
    original_f1_scores = metrics.classification_report(old_y_test, original_y_prediction, digits=3)

    score_file = fold_path + str(fold_index) + 'score_' + file_suffix + '.txt'
    with open(score_file, 'w') as f:
        f.write('fold: ' + str(fold_index) + '\n')
        f.write('scores: ' + str(scores) + '\n')
        f.write('scores with original testdata: ' + str(original_scores) + '\n')
        f.write('f1_scores:\n' + str(f1_scores) + '\n')
        f.write('f1_scores with original testdata:\n' + str(original_f1_scores) + '\n')

    loss_file_path = fold_path + str(fold_index) + 'loss_' + file_suffix + '.png'
    acc_file_path = fold_path + str(fold_index) + 'acc_' + file_suffix + '.png'
    plot_acc_loss(history.history, acc_file_path, loss_file_path)

    confusion_matrix_file_path = fold_path + str(fold_index) + 'confusion_matrix_' + file_suffix + '.png'
    plt_confusion_matrix(model, X_test, y_test, confusion_matrix_file_path)

In [16]:
def run_lstm(fold_index):
    old_test_file = 'CustomDataFull/testdata2000.pkl'
    train_file = 'folds/' + str(fold_index) + 'train_' + file_suffix
    test_file = 'folds/' + str(fold_index) + 'test_' + file_suffix
    valid_file = 'folds/' + str(fold_index) + 'valid_' + file_suffix

    X_train, y_train = get_data_from_file(train_file)
    X_test, y_test = get_data_from_file(test_file)
    X_valid, y_valid = get_data_from_file(valid_file)
    old_X_test, old_y_test = get_old_test_data(old_test_file)

    fold_path = 'results/lstm_results/' + global_folder_name + '/' + str(fold_index) + file_suffix + '/'
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)

    model = build_lstm_model()
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=global_learning_rate), metrics=['accuracy'])
    #model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=global_learning_rate), metrics=['accuracy', 
    #              tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tfa.metrics.F1Score(num_classes=10, average='macro',
    #              threshold=0.5)])
    
    model.summary()
    history = model.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid), batch_size=global_batch_size, epochs=global_epochs, shuffle=True, verbose=2)
    model.save(fold_path + str(fold_index) + 'model_' + file_suffix + '.h5')

    history_name = fold_path + str(fold_index) + 'history_' + file_suffix + '.history'

    with open(history_name, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

    scores = model.evaluate(X_test, y_test, verbose=1)
    y_prediction = model.predict(X_test)
    y_prediction = np.argmax(y_prediction, axis = 1)
    f1_scores = metrics.classification_report(y_test, y_prediction, digits=3)

    original_scores = model.evaluate(old_X_test, old_y_test, verbose=1)
    original_y_prediction = model.predict(old_X_test)
    original_y_prediction = np.argmax(original_y_prediction, axis = 1)
    original_f1_scores = metrics.classification_report(old_y_test, original_y_prediction, digits=3)
    
    score_file = fold_path + str(fold_index) + 'score_' + file_suffix + '.txt'
    with open(score_file, 'w') as f:
        f.write('fold: ' + str(fold_index) + '\n')
        f.write('scores: ' + str(scores) + '\n')
        f.write('scores with original testdata: ' + str(original_scores) + '\n')
        f.write('f1_scores: ' + str(f1_scores) + '\n')
        f.write('f1_scores with original testdata: ' + str(original_f1_scores) + '\n')

    loss_file_path = fold_path + str(fold_index) + 'loss_' + file_suffix + '.png'
    acc_file_path = fold_path + str(fold_index) + 'acc_' + file_suffix + '.png'
    plot_acc_loss(history.history, acc_file_path, loss_file_path)

    confusion_matrix_file_path = fold_path + str(fold_index) + 'confusion_matrix_' + file_suffix + '.png'
    plt_confusion_matrix(model, X_test, y_test, confusion_matrix_file_path)

### Hyperparameter tuning

In [17]:
def tune_cnn():
    """
    OBS! Efter en körning så printas summaryn över resultaten gällande de olika hyperparametrarna. Spara dom!
    """
    tuner = keras_tuner.RandomSearch(hyperparameter_cnn_model, overwrite=True, objective='val_accuracy', max_trials=3, executions_per_trial=3, directory='hyperparameters', project_name=global_hyperparameter_folder_name)
    tuner.search_space_summary()
    fold_index = 0 # Godtyckligt val!!!

    train_file = 'folds/' + str(fold_index) + 'train_' + file_suffix
    test_file = 'folds/' + str(fold_index) + 'test_' + file_suffix
    valid_file = 'folds/' + str(fold_index) + 'valid_' + file_suffix

    X_train, y_train = get_data_from_file(train_file)
    X_test, y_test = get_data_from_file(test_file)
    X_valid, y_valid = get_data_from_file(valid_file)
    tuner.search(X_train, y_train, epochs=global_epochs, batch_size=global_batch_size, validation_data=(X_valid, y_valid), shuffle=True, verbose=2, callbacks=[tf.keras.callbacks.TensorBoard(log_dir='hyperparameters/' + global_hyperparameter_folder_name)])
    
    print(tuner.get_best_hyperparameters()[0].values)

    hyperparameter_log_name = 'hyperparameters/' + global_hyperparameter_folder_name + '/log.txt'

    with open(hyperparameter_log_name, 'w') as f:
        f.write('best hyperparameters: ' + str(tuner.get_best_hyperparameters()[0].values) + '\n')
        f.write(str(tuner.results_summary()) + '\n')

    tuner.results_summary()

    copy_file = 'hyperparameters/' + global_hyperparameter_folder_name + '/' + 'cnn-hyperparameter_model.ipynb'
    own_file_name = own_file_path + '/k-fold_cross_validation.ipynb'
    shutil.copyfile(own_file_name, copy_file)

In [18]:
def tune_lstm():
    """
    OBS! Efter en körning så printas summaryn över resultaten gällande de olika hyperparametrarna. Spara dom!
    """
    tuner = keras_tuner.RandomSearch(hyperparameter_lstm_model, overwrite=True, objective='val_accuracy', max_trials=3, executions_per_trial=3, directory='hyperparameters', project_name=global_hyperparameter_folder_name)
    tuner.search_space_summary()
    fold_index = 0 # Godtyckligt val!!!

    train_file = 'folds/' + str(fold_index) + 'train_' + file_suffix
    test_file = 'folds/' + str(fold_index) + 'test_' + file_suffix
    valid_file = 'folds/' + str(fold_index) + 'valid_' + file_suffix

    X_train, y_train = get_data_from_file(train_file)
    X_test, y_test = get_data_from_file(test_file)
    X_valid, y_valid = get_data_from_file(valid_file)
    tuner.search(X_train, y_train, epochs=global_epochs, batch_size=global_batch_size, validation_data=(X_valid, y_valid), shuffle=True, verbose=2, callbacks=[tf.keras.callbacks.TensorBoard(log_dir='hyperparameters/' + global_hyperparameter_folder_name)])
    
    print(tuner.get_best_hyperparameters()[0].values)

    hyperparameter_log_name = 'hyperparameters/' + global_hyperparameter_folder_name + '/log.txt'

    with open(hyperparameter_log_name, 'w') as f:
        f.write('best hyperparameters: ' + str(tuner.get_best_hyperparameters()[0].values) + '\n')
    
    tuner.results_summary()
    
    copy_file = 'hyperparameters/' + global_hyperparameter_folder_name + '/' + 'lstm-hyperparameter_model.ipynb'
    own_file_name = own_file_path + '/k-fold_cross_validation.ipynb'
    shutil.copyfile(own_file_name, copy_file)

### Run 5-fold cross validation

In [19]:
def run_cnn_5_fold_cross_validation():
    path = 'results/cnn_results/' + global_folder_name
    if not os.path.exists(path):
        os.makedirs(path)

    copy_file = 'results/cnn_results/' + global_folder_name + '/' + 'copied_model.ipynb'
    own_file_name = own_file_path + '/k-fold_cross_validation.ipynb'
    shutil.copyfile(own_file_name, copy_file)

    for i in range(0, 5):
        run_cnn(i)

In [20]:
def run_lstm_5_fold_cross_validation():
    path = 'results/lstm_results/' + global_folder_name
    if not os.path.exists(path):
        os.makedirs(path)
    
    copy_file = 'results/lstm_results/' + global_folder_name + '/' + 'copied_model.ipynb'
    own_file_name = own_file_path + '/k-fold_cross_validation.ipynb'
    shutil.copyfile(own_file_name, copy_file)

    for i in range(0, 5):
        run_lstm(i)

In [21]:
if (global_model_type == 'cnn'):
    print('Start running cnn')
    run_cnn_5_fold_cross_validation()
elif (global_model_type == 'lstm'):
    print('Start running lstm')
    run_lstm_5_fold_cross_validation()
elif (global_model_type == 'tune_cnn'):
    print('Start tuning cnn')
    tune_cnn()
elif (global_model_type == 'tune_lstm'):
    print('Start tuning lstm')
    tune_lstm()

Trial 2 Complete [00h 49m 59s]
val_accuracy: 0.9689904650052389

Best val_accuracy So Far: 0.9787428577740988
Total elapsed time: 01h 43m 23s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
0.5               |0.1               |dropout_1
128               |128               |conv_2_filters_1
64                |128               |conv_2_filters_2

Epoch 1/50


2023-05-03 15:44:40.645437: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2188/2188 - 22s - loss: 2.0398 - accuracy: 0.2299 - val_loss: 1.6074 - val_accuracy: 0.4214 - 22s/epoch - 10ms/step
Epoch 2/50
2188/2188 - 20s - loss: 1.4742 - accuracy: 0.4641 - val_loss: 1.2528 - val_accuracy: 0.5280 - 20s/epoch - 9ms/step
Epoch 3/50
2188/2188 - 20s - loss: 1.2110 - accuracy: 0.5571 - val_loss: 0.9988 - val_accuracy: 0.6333 - 20s/epoch - 9ms/step
Epoch 4/50
2188/2188 - 20s - loss: 1.0684 - accuracy: 0.6077 - val_loss: 0.8970 - val_accuracy: 0.6731 - 20s/epoch - 9ms/step
Epoch 5/50
2188/2188 - 20s - loss: 0.9747 - accuracy: 0.6426 - val_loss: 0.7896 - val_accuracy: 0.7147 - 20s/epoch - 9ms/step
Epoch 6/50
2188/2188 - 20s - loss: 0.9017 - accuracy: 0.6698 - val_loss: 0.7965 - val_accuracy: 0.7110 - 20s/epoch - 9ms/step
Epoch 7/50
2188/2188 - 20s - loss: 0.8400 - accuracy: 0.6931 - val_loss: 0.6581 - val_accuracy: 0.7665 - 20s/epoch - 9ms/step
Epoch 8/50
2188/2188 - 20s - loss: 0.7838 - accuracy: 0.7124 - val_loss: 0.6123 - val_accuracy: 0.7798 - 20s/epoch - 9ms/step
Ep

2023-05-03 16:00:53.424406: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2188/2188 - 21s - loss: 2.0374 - accuracy: 0.2271 - val_loss: 1.6300 - val_accuracy: 0.3990 - 21s/epoch - 9ms/step
Epoch 2/50
2188/2188 - 19s - loss: 1.5176 - accuracy: 0.4472 - val_loss: 1.2450 - val_accuracy: 0.5489 - 19s/epoch - 9ms/step
Epoch 3/50
2188/2188 - 19s - loss: 1.2433 - accuracy: 0.5437 - val_loss: 1.0353 - val_accuracy: 0.6253 - 19s/epoch - 9ms/step
Epoch 4/50
2188/2188 - 19s - loss: 1.1129 - accuracy: 0.5912 - val_loss: 0.9543 - val_accuracy: 0.6554 - 19s/epoch - 9ms/step
Epoch 5/50
2188/2188 - 19s - loss: 1.0199 - accuracy: 0.6235 - val_loss: 0.8398 - val_accuracy: 0.6970 - 19s/epoch - 9ms/step
Epoch 6/50
2188/2188 - 19s - loss: 0.9509 - accuracy: 0.6491 - val_loss: 0.7640 - val_accuracy: 0.7200 - 19s/epoch - 9ms/step
Epoch 7/50
2188/2188 - 19s - loss: 0.8865 - accuracy: 0.6754 - val_loss: 0.7296 - val_accuracy: 0.7373 - 19s/epoch - 9ms/step
Epoch 8/50
2188/2188 - 19s - loss: 0.8338 - accuracy: 0.6953 - val_loss: 0.6685 - val_accuracy: 0.7611 - 19s/epoch - 9ms/step
Epo